<h1>Data Preparation</h1>

<h2>Importing pandas module</h2>

In [1]:
import pandas as pd

<h2>Reading and combining data</h2>

<h3>Creating a list that contains csv file names</h3>

In [2]:
csv_files = [
    "Bio-Diesel_PassengerVehicle_Stats.csv",
    "Compressed Natural Gas_PassengerVehicle_Stats.csv",
    "Diesel_PassengerVehicle_Stats.csv",
    "Electric_PassengerVehicle_Stats.csv",
    "Flex Fuel_PassengerVehicle_Stats.csv",
    "Gasoline_PassengerVehicle_Stats.csv",
    "Horse_PassengerVehicle_Stats.csv",
    "Hybrid_PassengerVehicle_Stats.csv",
    "Pedal_PassengerVehicle_Stats.csv",
]

<h3>Loading all 9 csv files into a list</h3>

In [3]:
# Load all files into a list
file_list = [] 
for file in csv_files:
    try:
        df = pd.read_csv(file)
        file_list.append(df)
        print(f"Loaded {file} successfully.")
    except FileNotFoundError:
        print(f"File {file} not found. Please check the file path.")
    except Exception as e:
        print(f"An error occurred while loading {file}: {e}")
print(len(file_list))

Loaded Bio-Diesel_PassengerVehicle_Stats.csv successfully.
Loaded Compressed Natural Gas_PassengerVehicle_Stats.csv successfully.
Loaded Diesel_PassengerVehicle_Stats.csv successfully.
Loaded Electric_PassengerVehicle_Stats.csv successfully.
Loaded Flex Fuel_PassengerVehicle_Stats.csv successfully.
Loaded Gasoline_PassengerVehicle_Stats.csv successfully.
Loaded Horse_PassengerVehicle_Stats.csv successfully.
Loaded Hybrid_PassengerVehicle_Stats.csv successfully.
Loaded Pedal_PassengerVehicle_Stats.csv successfully.
9


<h3>Concatenate files and Display the dataframe structure</h3>

In [4]:
#Display Dataframe Structucture and summary
vehicles_df = pd.concat(file_list, ignore_index = True)
print(vehicles_df.head())
print(vehicles_df.describe())
print(vehicles_df.info())

   Unnamed: 0  Public Vehicle Number    Status Vehicle Make Vehicle Model  \
0        1286                  12009  RESERVED    CHEVROLET       EXPRESS   
1        2095                  12248  INACTIVE     MERCEDES      SPRINTER   
2        7950                  13527  INACTIVE     VAN HOOL         TD925   
3        8700                  12248  INACTIVE     MERCEDES      SPRINTER   
4        9359                  13528  INACTIVE     VAN HOOL         TD925   

   Vehicle Model Year Vehicle Color Vehicle Fuel Source Wheelchair Accessible  \
0              2014.0         BLACK          Bio-Diesel                     N   
1              2010.0        SILVER          Bio-Diesel                     N   
2              2008.0           RED          Bio-Diesel                     N   
3              2010.0        SILVER          Bio-Diesel                     N   
4              2008.0           RED          Bio-Diesel                     N   

                              Company Name        

<h3>Remove the Duplicates</h3>

In [5]:
#Remove duplicates
vehicles_df.drop_duplicates(inplace=True)
#Check null values
print(vehicles_df.isnull().sum)

<bound method DataFrame.sum of        Unnamed: 0  Public Vehicle Number  Status  Vehicle Make  Vehicle Model  \
0           False                  False   False         False          False   
1           False                  False   False         False          False   
2           False                  False   False         False          False   
3           False                  False   False         False          False   
4           False                  False   False         False          False   
...           ...                    ...     ...           ...            ...   
16597       False                  False   False         False          False   
16598       False                  False   False         False          False   
16599       False                  False   False         False          False   
16600       False                  False   False         False          False   
16601       False                  False   False         False          False 

<h3> Droping the colomns with null values </h3>
   <ul>
       <li>Columns with more than 50% missing values often lack sufficient information to be meaningful or useful in analysis.</li>
   </ul>

In [6]:
threshold = 0.5  # Set the threshold for dropping columns
vehicles_df = vehicles_df.loc[:, vehicles_df.isnull().mean() < threshold]
print("Dropped columns with more than 50% null values.")
print(vehicles_df.info())

Dropped columns with more than 50% null values.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16602 entries, 0 to 16601
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             16602 non-null  int64  
 1   Public Vehicle Number  16602 non-null  int64  
 2   Status                 16602 non-null  object 
 3   Vehicle Make           14685 non-null  object 
 4   Vehicle Model          14639 non-null  object 
 5   Vehicle Model Year     14660 non-null  float64
 6   Vehicle Color          14616 non-null  object 
 7   Vehicle Fuel Source    16602 non-null  object 
 8   Wheelchair Accessible  16602 non-null  object 
 9   Company Name           16602 non-null  object 
 10  Address                14816 non-null  object 
 11  City                   14816 non-null  object 
 12  State                  14816 non-null  object 
 13  ZIP Code               14816 non-null  float64
 14  Record

<h3>Identify Outliers</h3>
 <ul>
       <li>Clean the data by removing outliers because outliers can distort statistical analysis </li>
   </ul>

In [7]:
import numpy as np
for column in vehicles_df.select_dtypes(include=np.number).columns:
    Q1 = vehicles_df[column].quantile(0.25)
    Q3 = vehicles_df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    vehicles_df.loc[(vehicles_df[column] < lower_bound) | (vehicles_df[column] > upper_bound), column] = np.nan

<ul>
    <li>Filling missing numeric values with median and missing categorical values with mode</li>
</ul>

In [8]:
vehicles_df.fillna(vehicles_df.median(numeric_only=True), inplace=True)

for col in vehicles_df.select_dtypes(include=['object']).columns:
    vehicles_df[col].fillna(vehicles_df[col].mode()[0], inplace=True)

C:\Users\mkdha\AppData\Local\Temp\ipykernel_11852\3102936380.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicles_df[col].fillna(vehicles_df[col].mode()[0], inplace=True)


<h3> Extracts the first sequence of alphabetic characters from Record ID column and saves it in new column called Vehicle Type</h3>

In [9]:
vehicles_df['Vehicle Type'] = vehicles_df['Record ID'].str.extract(r'([A-Za-z]+)')
# Print the updated DataFrame
print(vehicles_df[['Record ID', 'Vehicle Type']].head())

                  Record ID Vehicle Type
0  12009Charter Sightseeing      Charter
1  12248Charter Sightseeing      Charter
2  13527Charter Sightseeing      Charter
3  12248Charter Sightseeing      Charter
4  13528Charter Sightseeing      Charter
